In [1]:
import numpy as np
import pandas as pd

In [2]:
pokemon = pd.read_csv('./data/pokemon.csv')
Personality_df = pd.read_csv('./data/Personality.csv')
skill_df = pd.read_csv('./data/pokemon_skill.csv')
pokemon.shape

(1072, 13)

In [3]:
df = pokemon.copy()
df.head()

,number,name,type1,type2,total,hp,attack,defense,sp_attack,sp_defense,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,3,Gigantamax Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False


In [4]:
df.drop(['number', 'total', 'generation', 'legendary'], axis=1, inplace=True)
df.head()

,name,type1,type2,hp,attack,defense,sp_attack,sp_defense,speed
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45
1,Ivysaur,Grass,Poison,60,62,63,80,80,60
2,Venusaur,Grass,Poison,80,82,83,100,100,80
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80
4,Gigantamax Venusaur,Grass,Poison,80,82,83,100,100,80


In [5]:
poke_name = 'Latios' # input("Who's that pokemon! : ")

In [6]:
poke_type, poke_stats = df[df['name'] == poke_name].iloc[:, 1:3], df[df['name'] == poke_name].iloc[:, 3:]
poke_type.values

array([['Dragon', 'Psychic']], dtype=object)

In [7]:
poke_stats

,hp,attack,defense,sp_attack,sp_defense,speed
464,80,90,80,130,110,110


In [14]:
skill_name = 'Draco Meteor' # input("Who's that pokemon! : ")
skill_type, skill_class, skill_dmg = skill_df[skill_df['name'] == skill_name].iloc[:, 1:].values[0]
skill_type, skill_class, skill_dmg

('Dragon', 'special', '130')

In [15]:
IV = [31, 0, 31, 31, 31, 31] # input()
EV = [6, 0, 0, 252, 0, 252]   # input()

In [16]:
Personality_name = 'Modest' #input('What is your pokemon's PERSONALITY! : ')
Personality = Personality_df[Personality_df['Personality'] == Personality_name].iloc[:, 1:]
Personality

,hp,attack,defense,sp_attack,sp_defense,speed
2,1,0.9,1.0,1.1,1.0,1.0


In [17]:
real_stats = []

real_stats.append(int(poke_stats.iloc[0, 0] + (IV[0]/2) + (EV[0]/8) + 60))

for i in range(1, 6):
    stat = (poke_stats.iloc[0, i] + (IV[i]/2) + (EV[i]/8) + 5) * Personality.iloc[:, i]
    stat = int(stat.values[0])
    real_stats.append(stat)

real_stats

[156, 85, 100, 200, 130, 162]

In [18]:
weight = 1

if skill_type in poke_type.values:
    weight *= 1.5

if skill_type not in poke_type.values:
    pass

weight

1.5

In [ ]:
# Additional weight(ex:item, Ability)
add_weight = [2] # list(map(int, input("Additional Weight? : ").split(',')))

for i in add_weight:
    weight *= i

In [20]:
if skill_class == 'physical':
    damage = int((real_stats[1] * int(skill_dmg)) * weight)

if skill_class == 'special':
    damage = int((real_stats[3] * int(skill_dmg)) * weight)

damage

39000

### making skill dataset

https://pokemondb.net/move/all

In [27]:
skill_data = pd.DataFrame(columns=['name', 'type', 'class', 'power'])
skill_data

,name,type,class,power


In [34]:
def give_list(ref):
    global soup
    elements = soup.select(ref)

    temp_list = []

    for element in elements:
        temp_list.append(element.text)

    return temp_list

In [49]:
from bs4 import BeautifulSoup
import requests

url = 'https://pokemondb.net/move/all'

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

skill_data['name'] = give_list('a.ent-name')
skill_data['type'] = give_list('a.type-icon')
elements = soup.select('td.cell-icon')

temp_list = []

for i, element in enumerate(elements):
    if i % 2 == 1:
        temp_list.append(element['data-filter-value'])

skill_data['class'] = temp_list
skill_data

,name,type,class,power
0,"10,000,000 Volt Thunderbolt",Electric,special,NaN
1,Absorb,Grass,special,NaN
2,Accelerock,Rock,physical,NaN
3,Acid,Poison,special,NaN
4,Acid Armor,Poison,status,NaN
...,...,...,...,...
910,Yawn,Normal,status,NaN
911,Zap Cannon,Electric,special,NaN
912,Zen Headbutt,Psychic,physical,NaN
913,Zing Zap,Electric,physical,NaN


In [50]:
elements = soup.select('td.cell-num')

temp_list = []

for i, element in enumerate(elements):
    if i % 4 == 0:
        temp_list.append(element.text)

skill_data['power'] = temp_list
skill_data

,name,type,class,power
0,"10,000,000 Volt Thunderbolt",Electric,special,195
1,Absorb,Grass,special,20
2,Accelerock,Rock,physical,40
3,Acid,Poison,special,40
4,Acid Armor,Poison,status,—
...,...,...,...,...
910,Yawn,Normal,status,—
911,Zap Cannon,Electric,special,120
912,Zen Headbutt,Psychic,physical,80
913,Zing Zap,Electric,physical,80


In [51]:
# skill_data.to_csv('C:/Users/040/Desktop/docker/pokemon/pokemon_skill.csv')